### Note : I [blogged](http://talhaoz.com/?p=634) about this study.

See [here](http://talhaoz.com/news/) for other iPython notebooks on this project.

Project (datasets and the source code) is available on [GitHub](https://github.com/oztalha/News-Commentary-Tweets-of-Elites)

In [1]:
%cd ..

C:\Users\Talha\Documents\WinPython3\projects\News-Commentary-Tweets-of-Elites


In [2]:
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
from mykeys import tw
import networkx as nx
import itertools

In [51]:
get_party_representation()

In [66]:
get_party_activity()

In [67]:
plot_MP_activity()

In [68]:
df = pd.read_csv('data/TR-tweeps-En.csv',encoding='utf-8')
x  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
df = df[df.profs.str.contains('|'.join(x))]
df

,twhandle,name,fcnt,profs
9,abdulkadiraksu1,Abdülkadir Aksu,68,"AKP, MP"
11,aberatconkar,Ahmet Berat Çonkar,2,"MP, AKP"
12,acaliskan40,Abdullah Çalışkan,22,"MP, AKP"
14,ackilic76,Akif Çağatay Kılıç,2,"AKP, MP"
21,adnankeskinCHP,Adnan Keskin,10,"MP, CHP"
22,afifd,Afif Demirkıran,10,"MP, AKP"
24,AhmetBOgutken,Ahmet Baha Öğütken,10,"MP, AKP"
35,Ahmet_Davutoglu,Ahmet Davutoğlu,71,"MP, AKP"
45,Akif_Hamzacebi,Akif Hamzaçebi,0,"MP, CHP"
64,aliozgunduz,Ali Özgündüz,14,"Legist, CHP, MP"


In [69]:
# group by titles
tweets = pd.read_csv('data/TR-tweets.csv',encoding='utf-8',parse_dates=['dt'])
tweets.twhandle = tweets.twhandle.str[1:]
title = tweets.groupby(by=['title','dt'])['twhandle']

In [70]:
# get monitored screen_names for each party
df = pd.read_csv('data/TR-tweeps-En.csv',encoding='utf-8')
groups  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
group = {}
mps = set()
for p in groups:
    group[p] = set(df[df.profs.str.contains(p)].twhandle.tolist())
    mps.update(group[p])

In [73]:
# get number of MPs of each party for each title
news = {}
def get_twbyp(g,partililer):
    tweeps = g.values.tolist()
    return len([t for t in tweeps if t in partililer])
    
for p in groups:
    news[p] = title.apply(lambda g: get_twbyp(g,group[p]))
    #news[p] = title.apply(lambda g: len(group[p] & set(g.values))) # number of MPs for the title

In [74]:
#sum all commentated titles up, 
polnews = pd.Series(index=news['Independent'].index,data=0)
for p in groups:
    print (news[p].sum(),'comments made on',news[p][news[p]>0].size,'news by',p)
    polnews = polnews.add(news[p])
print (polnews.sum(),'comments made on',polnews[polnews>0].size,'news by TBMM (any MP)')

6678 comments made on 2747 news by AKP
14228 comments made on 4925 news by CHP
3039 comments made on 2133 news by MHP
1140 comments made on 753 news by HDP
1915 comments made on 1471 news by Independent
27000 comments made on 6839 news by TBMM (any MP)


In [10]:
# for each party calculate diff = party - others
diff = {}
for p in groups:
    diff[p] = 2*news[p] - polnews

In [11]:
# the most commentated events by all MPS
polnews.order(ascending=False).head(60)

title                                                          dt        
İşte demokratikleşme paketi                                    2013-09-30    55
Berkin Elvan yaşam mücadelesini kaybetti                       2014-03-11    50
Öğretmenler günü kutlu olsun                                   2013-11-24    50
Mutlu yıllar Türkiye                                           2013-12-31    47
Cumhuriyet’in 90′ıncı yılı kutlu olsun                         2013-10-29    44
Büyük Zafer’in 91′inci yılı                                    2013-08-30    41
Ramazan ayı başladı                                            2013-07-09    40
Kemal Kılıçdaroğlu’na Meclis’te yumruklu saldırı               2014-04-08    39
Bugün Dünya Engelliler Günü                                    2013-12-03    37
Nice 19 Mayıslara…                                             2013-05-19    36
Darbenin adı: 12 Eylül 1980                                    2013-09-12    36
Bugün Dünya Engelliler Günü                   

In [12]:
# Number of commentations on events by AKP MPs - Other MPs
diff['AKP'].order(ascending=False)

title                                                          dt        
AK Parti’de Davutoğlu dönemi başladı                           2014-08-27    21
Başbakan Menderes anılıyor                                     2013-09-17    19
Genç’ten Emine Erdoğan protestosu                              2013-11-26    18
Marmaray açıldı; ilk sefer yapıldı                             2013-10-29    18
AK Parti 13′üncü yılını kutluyor                               2014-08-14    18
AK Parti 11 yıldır iktidarda                                   2013-11-03    16
AK Partililerden “DirenÇözüm” atağı                            2013-06-29    15
ODTÜ videosunun yankısı sürüyor                                2013-09-05    14
Başörtülü milletvekilleri Meclis’te                            2013-10-31    13
Adnan Menderes idam edilmesinin 53′üncü yılı                   2014-09-17    12
IMF ve dış borç tartışması dolu dizgin – nediyor.com           2013-05-14    12
Çankaya Köşkü’nde devir teslim gerçekleşti    

In [13]:
# Number of commentations on events by CHP MPs - Other MPs
diff['CHP'].order(ascending=False)

title                                                   dt        
Mustafa Balbay tahliye edildi                           2013-12-10    27
21 yıldır aydınlanmayan suikast                         2014-01-24    19
Deniz Gezmiş ve arkadaşları anıldı – nediyor.com        2013-05-06    19
CHP 90 yaşında                                          2013-09-09    18
Berkin Elvan yaşam mücadelesini kaybetti                2014-03-11    16
Ecevit 8′inci ölüm yıl dönümünde anılıyor               2014-11-05    16
Twitter yasağına tepkiler çığ gibi                      2014-03-21    16
Madımak Katliamı’nın 21′nci yıl dönümü                  2014-07-02    15
Karanlıkta 20′nci yıl – nediyor.com                     2013-01-24    15
CHP 91 yaşında                                          2014-09-09    15
Sarısülük’ü öldürmekle suçlanan polis serbest           2013-06-24    14
Sivas’ta yaşamını yitirenler anıldı                     2013-07-02    14
Ali İsmail ölümünün 1′inci yıl dönümünde anıldı         2

In [14]:
# Number of commentations on events by MHP MPs - Other MPs
diff['MHP'].order(ascending=False)

title                                                        dt        
Bahçeli’yi çılgına çeviren görüntü                           2013-06-15     5
Bahçeli’den Başbakan’a ağır sözler                           2013-06-22     5
MHP’den Ankara mitingi                                       2013-11-09     5
TRT spikerinden “yavru muhalefet” gafı                       2014-01-25     5
Bahçeli’den açık tehdit – nediyor.com                        2013-03-23     4
Eski TBMM Başkanvekili yaşamını yitirdi                      2014-06-21     4
Bahçeli Elazığ’da sert konuştu                               2013-09-15     4
Bahçeli’den Başbakan için ağır ifadeler                      2013-10-26     4
Başbakan’dan Vali Coş’a destek: “Yedirmeyiz”                 2013-11-12     4
“Türk ırkı yok” tartışması dinmiyor                          2013-12-02     4
Alparslan Türkeş ölümünün 17′inci yılında anıldı             2014-04-04     3
Tahliye edilen Alan : “Ne kadar namussuz varsa…”             2014-06-2

In [15]:
# Number of commentations on events by HDP MPs - Other MPs
diff['HDP'].order(ascending=False)

title                                                       dt        
Mardin ve Hakkari’de duvar gerginliği                       2013-11-07     5
“Demokrasi ve Barış Konferansı” sona erdi                   2013-05-25     5
“Hayata Dönüş” utancının 13′üncü yılı                       2013-12-19     5
Paris’teki 3′lü cinayetin üzerinden 2 yıl geçti             2015-01-09     4
3 PKK’lının cenazesi Diyarbakır’da – nediyor.com            2013-01-16     4
BDP yerel seçim stratejisini belirledi                      2013-09-09     4
Van Belediye Başkanı serbest – nediyor.com                  2013-03-29     3
HDP kongresinde Kürkçü kalp krizi geçirdi                   2013-10-27     3
Öcalan’ın son fotoğrafı Twitter’da                          2013-12-20     3
Ağrı’da oylar yeniden sayılıyor                             2014-04-04     3
Mehmet Öcalan serbest bırakıldı                             2013-08-04     3
Uludere raporu kabul edildi – nediyor.com                   2013-03-27     3
HDP e

In [16]:
# Number of commentations on events by Independent MPs - Other MPs
diff['Independent'].order(ascending=False)

title                                                       dt        
“Kimse Yok Mu”nun yetkisi elinden alındı                    2014-10-02     3
Cizre’de polise roketli saldırı: 2 yaralı                   2015-01-07     3
Ankara’da 13 polise “usulsüz taltif” iddiasıyla gözaltı     2014-09-22     3
İETT’den Cemaat hamlesi: Fatih Üni. hattı kaldırıldı        2014-09-23     3
İBB’nin gündemi Fatih Koleji: Bayramda yıkım                2014-10-06     3
“Kimse Yok Mu” derneği kapatılıyor mu?                      2014-09-29     3
Tuğluk: “PKK ta­ri­hi­nin en güç­lü dö­ne­mi­ni ya­şı­yor”  2014-08-26     2
PKK İzmir’de yol kesti                                      2014-05-30     2
IŞID’den petrol alındığı haberine Erdoğan’dan yalanlama     2014-09-16     2
Başbakan’a “Apo” tepkisi                                    2014-07-28     2
Saraç YÖK Başkanı’na hakaret mi etti?                       2014-02-22     2
PKK’lı Karasu: “AK Partili bakan adeta bize yalvardı”       2014-11-28     2
Saray

In [17]:
G=nx.Graph()
# add vertices
for p in groups:
    for politician in group[p]:
        G.add_node(politician, group=p)

In [18]:
def updateG(group,G,politicians):
    """ Create weighted edges """
    edges = itertools.combinations(group.tolist(), 2)
    for v1,v2 in edges:
        if v1 not in politicians or v2 not in politicians:
            continue
        if G.has_edge(v1, v2):
            G[v1][v2]['weight'] += 1
        else:
            G.add_edge(v1, v2, weight=1)

In [19]:
# add edges
title.apply(updateG,G,mps)
# print number of nodes and edges
print (G.number_of_nodes(),G.number_of_edges())

167 10236


In [22]:
# Exporting to be read by Gephi for better visualization
nx.write_gml(G,"data/nediyor_politics.gml")

See the interactive output here: [TBMM Co-commentation Network Visualization](http://talhaoz.com/news/tbmm)

In [28]:
# calculate the likelihood of tie formation among parties
# AKP-AKP, AKP-CHP, AKP-MHP, AKP-HDP, AKP-Bagimsiz
G = nx.read_gml("data/nediyor_politics.gml")

In [69]:
def get_likelihood(g1,g2):
    """ returns tie formation likelihood of group1 and group2"""
    groups = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
    likelihoods = []
    for politician in G:
        if G.node[politician]['group'] != g1:
            continue
        g2weight = .0 # sum of edge weights of nbors whose party is g2
        total = .0 # sum of all edge weights (all neighbors)
        for nbor in G[politician]:
            if G.node[nbor]['group'] == g2:
                g2weight += G[politician][nbor]['weight']
            total += G[politician][nbor]['weight']
        if total > 0:
            likelihoods.append(g2weight/total)
    l = sum(likelihoods)/len(likelihoods)
    print(g1+'-'+g2+' likelihood {:.2f}'.format(l))
    return l

In [72]:
groups  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
for g1 in groups:
    for g2 in groups:
        if groups.index(g1)>groups.index(g2):
            continue
        get_likelihood(g1,g2)

AKP-AKP likelihood 0.47
AKP-CHP likelihood 0.36
AKP-MHP likelihood 0.08
AKP-HDP likelihood 0.03
AKP-Independent likelihood 0.06
CHP-CHP likelihood 0.59
CHP-MHP likelihood 0.09
CHP-HDP likelihood 0.05
CHP-Independent likelihood 0.05
MHP-MHP likelihood 0.13
MHP-HDP likelihood 0.03
MHP-Independent likelihood 0.09
HDP-HDP likelihood 0.17
HDP-Independent likelihood 0.05
Independent-Independent likelihood 0.09


In [ ]:
#community detection
import 

In [65]:
def get_party_representation():
    """ Plot Tweep Rate by (i) MP counts, (ii) Monitored MP counts """

    #http://www.tbmm.gov.tr/develop/owa/milletvekillerimiz_sd.dagilim
    chairs = {'AKP':312, 'CHP':125, 'HDP':27, 'MHP':52, 'Independent':13}
        
    df = pd.read_csv('data/TR-tweeps-En.csv',encoding='utf-8')
    groups  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
    group = {}
    mps = set()
    for p in groups:
        group[p] = set(df[df.profs.str.contains(p)].twhandle.tolist())
        mps.update(group[p])
    
    y1=[int((len(group[p])*1.0/chairs[p])*100+0.5) for p in groups]
    y2=[len(group[p]) for p in groups]
    trace1 = Bar(x=groups, y=y1, name='Percentage of MPs normalized by # of chairs')
    trace2 = Bar(x=groups, y=y2, name='Plain number of MPs monitored')
    data = Data([trace1, trace2])
    
    layout = Layout(autosize=True,
        title='Members of TBMM monitored',
        barmode='group',
        legend=Legend(xanchor='right',yanchor='top', y=1,traceorder='normal'),
        yaxis=YAxis(title='MPs monitored by nediyor.com'),
        #yaxis2=YAxis(title='Percentage of MPs',side='right',overlaying='y',anchor='x')
    )
    fig = Figure(data=data, layout=layout)
    return py.iplot(fig,filename='MPs monitored by nediyor.com')


def get_party_activity():
    """ Plot Twitter Usage normalized by (i) MP counts, (ii) Monitored MP counts """

    # plot commentating activity of parties
    df = pd.read_csv('data/TR-tweeps-En.csv',encoding='utf-8')
    x  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
    df = df[df.profs.str.contains('|'.join(x))]
    mparty = dict(zip(df.twhandle, df.profs.map(lambda mp: [p for p in x if p in mp][0])))

    tweets = pd.read_csv('data/TR-tweets.csv',encoding='utf-8',parse_dates=['dt'])
    tweets['twhandle']=tweets['twhandle'].str[1:]
    tweets = tweets.rename(columns={'twtext':'twcnt'})
    tweeps = tweets.groupby(by='twhandle')['twcnt'].count().order(ascending=False)
    poltweeps = tweeps[tweeps.index.isin(df.twhandle)]
    
    vekil = pd.DataFrame.from_dict(mparty,orient='index')
    vekil.index.name = 'twhandle'
    vekil.columns = ['party']
    df = pd.DataFrame(poltweeps).join(vekil)
    
    #http://www.tbmm.gov.tr/develop/owa/milletvekillerimiz_sd.dagilim
    parties = {'AKP':312, 'CHP':125, 'HDP':27, 'MHP':52, 'Independent':13}
    mpc = sum(parties.values())
        
    # Parties' newsworthy tweet counts normalized by MP counts
    perMP = {}
    for p,cnt in parties.items():
        perMP[p] = df[df.party == p].twcnt.sum()/float(cnt)

    # Parties' newsworthy tweet counts normalized by MPs with Twitter accounts
    perTweep = {}
    for p,cnt in parties.items():
        perTweep[p] = df[df.party == p].twcnt.sum()/float(df[df.party == p].shape[0])
    
    
    x  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
    y1 = [int((perMP[p] * 100) + 0.5) / 100.0 for p in x]
    y2 = [int((perTweep[p] * 100) + 0.5) / 100.0 for p in x]
    #tweep/mp ratio
    # y = [df[df.profs.str.contains(p)].shape[0] for p in x]
    
    texts = []
    for p in x:
        twpset = poltweeps.index.tolist()
        text = u''
        for i,twp in enumerate(twpset):
            #twp = '<a href="http://twitter.com/'+twp[1:]+'">'+ twp +'</a>'
            text+= ' '+ twp if (i+1) % 10 != 0 else '<br>'+twp
        texts.append(text)
        
    trace1 = Bar(x=x, y=y1, name='normalized by MP counts') #, text = texts
    trace2 = Bar(x=x, y=y2, name='normalized by curated tweep counts')
    data = Data([trace1, trace2])
    layout = Layout(autosize=True,
        title="Newsworthy Tweet Counts",
        barmode='group',
        legend=Legend(x=0, y=1,traceorder='normal'),
        yaxis=YAxis(title='Number of newsworthy tweets per MP'),
        annotations=Annotations([
            Annotation(
            text='Code and datasets are available on <a href="https://github.com/oztalha/News-Commentary-Tweets-of-Elites">github.com/oztalha</a><br>'\
                 'Two years of curated tweets are scraped from <a href="http://nediyor.com/">nediyor.com</a>',
            showarrow=False,
            x=0.01,
            y=0.85,
            xref='paper',
            yref='paper',
            xanchor='left',
            yanchor='top',
            align='left'
            ),
        ]))
    fig = Figure(data=data, layout=layout)
    return py.iplot(fig, filename='Newsworthy Tweet Counts of Turkish Parties')


def plot_MP_activity():
    # plot commentating activity of these politicians
    df = pd.read_csv('data/TR-tweeps-En.csv',encoding='utf-8')
    x  = ['AKP', 'CHP', 'MHP', 'HDP','Independent']
    df = df[df.profs.str.contains('|'.join(x))]
    mparty = dict(zip(df.twhandle, df.profs.map(lambda mp: [p for p in x if p in mp][0])))

    tweets = pd.read_csv('data/TR-tweets.csv',encoding='utf-8',parse_dates=['dt'])
    tweets['twhandle']=tweets['twhandle'].str[1:]
    tweeps = tweets.groupby(by='twhandle')['twtext'].count().order(ascending=False)
    poltweeps = tweeps[tweeps.index.isin(df.twhandle)]
    partycolors = {'AKP':'orange', 'CHP':'red', 'MHP':'green', 'HDP':'purple','Independent':'cyan'}
    colors = [partycolors[mparty[x]] for x in poltweeps.index]
    data = Data([Bar(
            x=poltweeps.index,
            y=poltweeps.values,
            marker=Marker(color=colors)
            )])
    layout = Layout(autosize=True, margin=Margin(b=111),
                    yaxis=YAxis(title='# of news commentated (Jan 2013 - Jan 2015)'),
                    title="News commentated by Members of TBMM (curated by nediyor.com)")
    fig = Figure(data=data, layout=layout)
    return py.iplot(fig,filename="TBMM as News Commentators")

In [13]:
    tweets = pd.read_csv('data/TR-tweets.csv',encoding='utf-8',parse_dates=['dt'])
    tweets['twhandle']=tweets['twhandle'].str[1:]
    tweeps = tweets.groupby(by='twhandle')['twtext'].count().order(ascending=False)
    poltweeps = tweeps[tweeps.index.isin(df.twhandle)]

In [77]:
for k,v in mparty.items():
    if v == 'Independent':
        print(k)

idrisbal
Mv_MCetin
hakansukur
ErtugrulGunay


In [49]:
    for p,cnt in parties.items():
        print(p,df[df.party == p].twcnt.sum(), df[df.party == p].shape[0])

Independent 1915 4
MHP 3039 13
HDP 1139 17
CHP 14205 58
AKP 6677 74
